In [1]:
import numpy as np
import metropolishastings as mh

We use the Metropolis-Hastings algorithm to draw from a discrete random variable $X$ that takes 4 values, with the following pmf:

- $P(X=1) = 20 / K$,
- $P(X=2) = 8 / K$,
- $P(X=3) = 3 / K$,
- $P(X=4) = 1 / K$,

where $K$ is the normalization factor that is such that the probabilities sum up to one. In this simple example, the value of $K$ can be calculated ($K=32$). But the point is that the MH algorithm does not need it. It works with unnormalized probabilities.

The generating Markov process is a random walk jumping from one state to the next with equal probability.

In [2]:
def myprocess(state):
    next_state = np.random.randint(1, 4+1)
    return next_state, np.log(0.25), np.log(0.25)

The weights are the unnormalized probabilities

In [3]:
weights = [20, 8, 3, 1]

In [4]:
def logweight(state):
    w = {
        i+1: np.log(w) for i, w in enumerate(weights)
    }
    return w[state]

The indicators are the parameters of interest, generated by each draw. In this case, it is a vector of all zeros, except one for the state corresponding to the draw. Calculating the mean of these indicators gives the average number of times that the Markov process has visited each state.

In [5]:
def indicators(one_draw):
    result = [1 if (one_draw == i) else 0 for i in [1, 2, 3, 4]]
    return result

We start four independent Markov processes, each staring from a different state.

In [6]:
initialStates = [1, 2, 3, 4]
numberOfDraws = 100000
maxNumberOfIterations = 10

We apply the MH algorithm.

In [7]:
draws, estimates, convergence, numberOfTrials = mh.MetropolisHastings(
    initialStates,
    myprocess,
    logweight,
    indicators,
    numberOfDraws,
    maxNumberOfIterations,
)

Success rate: 0.51614
Success rate: 0.51458
Success rate: 0.51496
Success rate: 0.51353


Here are the estimates for the frequency of each state.

In [8]:
estimates

array([0.6274375, 0.2485175, 0.0929325, 0.0311125])

We calculate the theoretical probabilities for comparison.

In [9]:
prob = np.array(weights)
prob = prob / prob.sum()
prob

array([0.625  , 0.25   , 0.09375, 0.03125])

With this simple example, the MArkov porcess had apparently reached stationarity with the first set of draws that was generated. Apparently, because the criteria used are heuristics, and not exact. 

In [10]:
numberOfTrials

1

Here are the draws

In [11]:
draws

array([[0, 0, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
draws.shape

(4, 400000)

We can verify the the mean of each series corresponds to the estimates reported above

In [13]:
draws[0].mean()

0.6274375

In [14]:
draws[1].mean()

0.2485175

In [15]:
draws[2].mean()

0.0929325

In [16]:
draws[3].mean()

0.0311125